## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-13-09-32-09 +0000,nyt,Nepal’s New Government Calls Elections. Its Cr...,https://www.nytimes.com/2025/09/13/world/asia/...
1,2025-09-13-09-30-35 +0000,nypost,California legislature passes bill prohibiting...,https://nypost.com/2025/09/13/us-news/californ...
2,2025-09-13-09-30-00 +0000,wsj,Big Brands Play Down Their Americanness Abroad,https://www.wsj.com/business/retail/big-brands...
3,2025-09-13-09-02-49 +0000,nyt,Brazil Defies Trump’s Demands by Convicting Bo...,https://www.nytimes.com/2025/09/13/world/ameri...
4,2025-09-13-09-02-41 +0000,nyt,The Police Found Messages After Kirk’s Killing...,https://www.nytimes.com/2025/09/13/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2422/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
34,kirk,52
121,charlie,46
27,trump,26
349,shooting,14
1,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
83,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,147
16,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...,147
56,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...,147
133,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,144
46,2025-09-13-03-51-29 +0000,nyt,"Tyler Robinson, Charlie Kirk Shooting Suspect,...",https://www.nytimes.com/2025/09/12/us/politics...,144


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
83,147,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...
16,49,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...
75,43,2025-09-13-00-40-07 +0000,nypost,Charlie Kirk’s wife Erika delivers emotional f...,https://nypost.com/2025/09/12/us-news/charlie-...
123,41,2025-09-12-21-08-55 +0000,nypost,Ukraine strikes key Russian oil terminal in la...,https://nypost.com/2025/09/12/world-news/ukrai...
129,40,2025-09-12-21-01-59 +0000,bbc,Watch: BBC reports from residence linked to Ki...,https://www.bbc.com/news/videos/cq65y02047lo?a...
229,39,2025-09-12-14-43-17 +0000,nypost,Warner Bros. Discovery CEO David Zaslav wants ...,https://nypost.com/2025/09/12/business/warner-...
217,39,2025-09-12-15-18-57 +0000,startribune,Another shocking act of political violence lea...,https://www.startribune.com/charlie-kirk-minne...
298,37,2025-09-12-10-00-00 +0000,latimes,California lawmakers pass bill banning law enf...,https://www.latimes.com/california/story/2025-...
212,37,2025-09-12-15-47-11 +0000,nypost,Ex-White House press secretary Karine Jean-Pie...,https://nypost.com/2025/09/12/us-news/ex-white...
197,34,2025-09-12-17-08-00 +0000,bbc,Anti-apartheid hero's new inquest should lead ...,https://www.bbc.com/news/articles/c931n9eelpeo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
